This notebook gives a quick demo of PyJulia which allows you to use Julia from within Python.  For more details and further examples, see the PyJulia [documentation](https://pyjulia.readthedocs.io/en/latest/).

# Setup

Running this notebook requires the following:
* jupyter
* python 3
* julia

To run on your local machine, do the following:
1. **Install Anaconda**: The anaconda installer can be downloaded [here](https://www.anaconda.com/products/individual) (at the bottom of the page). Follow the install instructions.
2. **Install Julia**: Prebuilt Julia binaries can be downloaded [here](https://julialang.org/) or the source code cloned from [here](https://github.com/JuliaLang/julia/tree/v1.5.1).  Follow the install or build instructions.
3. **Create an environment with python 3 and jupyter**: Open a terminal and type
```
conda create -n pyjulia python=3 jupyter numpy
conda activate pyjulia
jupyter notebook PyJulia_Demo
```
4. **Run the cells under Install PyJulia**.

To run on Eagle:
1. See the instruction [here](https://github.com/NREL/HPC/blob/master/languages/python/jupyter/Kernels_and_Servers.ipynb) for running jupyter notebooks on Eagle.
2. See the instruction [here](../../how_to_guides/build_Julia.md) for building Julia on Eagle.
3. Run the cells under Install PyJulia.

# Install PyJulia

Run the following cells to install the julia python module.  For more detailed installation instructions, see the PyJulia [install instructions](https://pyjulia.readthedocs.io/en/latest/installation.html#step-2-install-pyjulia).

You may skip these if the julia python module is already installed.

In [ ]:
!pip install --user julia

In [ ]:
import julia
julia.install()

# Demo

Conda's python is statically linked to libpython which causes issues with Julia.  To get around this, we start julia without using compiled modules.

In [ ]:
from julia.api import Julia
jl = Julia(compiled_modules=False)

We can evaluate any Julia statement by calling `eval`.

**NOTE:**  Any print statements given to julia will print to the terminal that you launched jupyter from rather than here in the notebook.

In [ ]:
 jl.eval("println(\"Hello World! I'm printing using eval!\")")

The `eval` function returns whatever is returned by the julia expression in the string.

In [ ]:
jl.eval('5 + 3*2')

We can also import julia packages and access them directly from python.  Again, we are calling a julia side print so it will print to the terminal and not the notebook.

In [ ]:
from julia import Base # make julia Base module python callable
Base.println("Hello World! I'm printing directly from python!")

You can access the julia help for a function.

**NOTE:** As of this writing, `help` is broken.  Below should be the help for sum but generates an error about `Markdown` not being defined. If that is package is installed, and we do `jl.using("Markdown")` then the help for one of the `String` constructors is returned which is not correct.

In [ ]:
jl.help("Base.sum") # BROKEN!!!

We can activate a Julia environment and even instantiate it.  Julia's activation message will appear in the terminal.

In [ ]:
path = "." # path to desired Julia Project file
jl.using("Pkg")
from julia import Pkg
Pkg.activate(path)
Pkg.instantiate()

Julia's global scope is available from the Main package.  Importing it allows us access anything in it directly from python.

In [ ]:
from julia import Main
Main.log(Main.exp(2))

We can also create things in this scope

In [ ]:
x = 5.25 # python variable
Main.z = 1.5 * x 

We can now use `z` variable in Julia

In [ ]:
jl.eval("z / 1.5")

We can do this with more complex types as well

In [ ]:
py_dict = {'a':1,'b':2,'c':3}
Main.jl_dict = py_dict
print(Main.string(Main.jl_dict))
print(py_dict)

In [ ]:
import numpy as np
t = np.random.uniform(size=10)
Main.t = t + 1.0
print(Main.typeof(Main.t))
print(Main.t)

**WARNING:** Julia types can change when going through python.

In [ ]:
print(jl.eval("typeof(Int32(5))")) # Int32
print(Main.typeof(Main.Int32(5))) # Int64

Python functions can also be called from Julia

In [ ]:
# Define a python function
def my_function(arg):
    return 5*arg

# Make the above function visible in julia
Main.my_function=my_function
# Call directly
print(Main.my_function(5.0))
print(Main.my_function('q')) # Note we get python behavior and not julia--in julia this would throw an error

# Create a julia function which calls the python function after converting the argument to a string
jl.eval('function test(input::Any); return my_function(string(input)); end;')

# Call it--note we still get python behavior and not julia behavior
print(Main.test('s'))
print(Main.test(5))
print(Main.test(5.0))

In [ ]:
def python(arg):
    return 'My arg is "{}"'.format(arg)

def monty(arg1, arg2):
    return 'Black Knight: {}\nArthur: {}'.format(arg1,arg2)

import numpy as np
def do_math(arg):
    return np.exp(np.pi * arg)

jl.eval('function call_it(f, args...); return f(args...); end;')
print(Main.call_it(python, 'ARG!!'))
print(Main.call_it(monty, "'Tis but a flesh wound!", "A flesh wound! Your arms off!"))
print(Main.call_it(do_math, -1.0))

We can also use more complicated Julia stuff like JuMP.  However, python doesn't like some of julia's exotic syntax (e.g. `@` for macro calls and `!` in function calls.

In [ ]:
# Build and solve a JuMP model in julia with Ipopt
jl.using("JuMP")
from julia import JuMP

model = JuMP.Model()
Main.model = model # Make `model` a variable that julia can see

jl.eval("@variable(model, x >= 0.0)")
jl.eval("@variable(model, y >= 0.0)")
jl.eval("@objective(model, Min, (x - 6)^2 + 0.1*x*y + (y - 7)^2)")
jl.eval("@constraint(model, x + y == 10.0)")

jl.using("Ipopt")
jl.eval('JuMP.set_optimizer(model, ()->Ipopt.Optimizer())') # Output from IPOPT will print to the terminal
jl.eval('JuMP.optimize!(model)')

print(JuMP.termination_status(model))
print(JuMP.objective_value(model))
print(JuMP.value(Main.x)) # Use Main.x instead of x b/c x is not visible to python
print(JuMP.value(Main.y)) # Same as x